In [3]:
#!pip install kserve==0.7.0

In [4]:
from kubernetes import client 
from kserve import KServeClient
from kserve import constants
from kserve import utils
from kserve import V1beta1InferenceService
from kserve import V1beta1InferenceServiceSpec
from kserve import V1beta1PredictorSpec
from kserve import V1beta1SKLearnSpec

/opt/conda/lib/python3.8/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  warnings.warn(


In [6]:
namespace = utils.get_default_target_namespace()

In [12]:
name='sklearn-iris-2'
kserve_version='v1beta1'
api_version = constants.KSERVE_GROUP + '/' + kserve_version

isvc = V1beta1InferenceService(api_version=api_version,
                               kind=constants.KSERVE_KIND,
                               metadata=client.V1ObjectMeta(
                                   name=name, namespace=namespace, annotations={'sidecar.istio.io/inject':'false'}),
                               spec=V1beta1InferenceServiceSpec(
                               predictor=V1beta1PredictorSpec(
                               sklearn=(V1beta1SKLearnSpec(
                                   storage_uri="gs://kfserving-samples/models/sklearn/iris"))))
)

In [13]:
KServe = KServeClient()
KServe.create(isvc)

{'apiVersion': 'serving.kserve.io/v1beta1',
 'kind': 'InferenceService',
 'metadata': {'annotations': {'sidecar.istio.io/inject': 'false'},
  'creationTimestamp': '2022-03-18T17:31:37Z',
  'generation': 1,
  'managedFields': [{'apiVersion': 'serving.kserve.io/v1beta1',
    'fieldsType': 'FieldsV1',
    'fieldsV1': {'f:metadata': {'f:annotations': {'.': {},
       'f:sidecar.istio.io/inject': {}}},
     'f:spec': {'.': {},
      'f:predictor': {'.': {}, 'f:sklearn': {'.': {}, 'f:storageUri': {}}}}},
    'manager': 'OpenAPI-Generator',
    'operation': 'Update',
    'time': '2022-03-18T17:31:34Z'}],
  'name': 'sklearn-iris-2',
  'namespace': 'kubeflow-user-example-com',
  'resourceVersion': '213235',
  'selfLink': '/apis/serving.kserve.io/v1beta1/namespaces/kubeflow-user-example-com/inferenceservices/sklearn-iris-2',
  'uid': '7350a563-2c5b-4a06-88e1-f649ad18db1e'},
 'spec': {'predictor': {'sklearn': {'name': 'kserve-container',
    'protocolVersion': 'v1',
    'resources': {'limits': {'

In [14]:
KServe.get(name, namespace=namespace, watch=True, timeout_seconds=120)

NAME                 READY                           PREV                    LATEST URL                                                              
sklearn-iris-2       True                               0                       100 http://sklearn-iris-2.kubeflow-user-example-com.example.com      


In [15]:
import requests

isvc_resp = KServe.get(name, namespace=namespace)
isvc_url = isvc_resp['status']['address']['url']

print(isvc_url)

inference_input = {
  'instances': [
    [6.8,  2.8,  4.8,  1.4],
    [6.0,  3.4,  4.5,  1.6]
  ]
}

response = requests.post(isvc_url, json=inference_input)
print(response.text)

http://sklearn-iris-2.kubeflow-user-example-com.svc.cluster.local/v1/models/sklearn-iris-2:predict
{"predictions": [1, 1]}


In [16]:
print('END OF NOTEBOOK')

END OF NOTEBOOK
